In [1]:
# !pip install requests
# !pip install html5lib
# !pip install beautifulsoup4
# !pip install pandas
# !pip install numpy

import pandas as pd
from time import sleep
from random import randint
import numpy as np
import requests
from bs4 import BeautifulSoup
import pickle

def web_scrape(url_link,start,end):  
    headlines = []
    Published = []
    pages = np.arange(start, end, 1)
 
    for page in pages:
        url = url_link + str(page)
        r = requests.get(url,headers = {'User-Agent': 'Mozilla/5.0 '}).text
        soup1 = BeautifulSoup(r)
        # print('Page No:{}'.format(page))
        count = 0
        for link in soup1.find_all('a',{'class':'title'}, href=True, title=True):
            for date in soup1.find_all("span", class_="date"):
                sleep(randint(2, 10))
                count = count+1
                links = [link['title']]
                dates = [date.text]
                #print(links,dates) 
                for lines in links:
                    if count > 6 and count <= 16:     #To avoid the noise/repetitive headlines                   
                        headlines.append(lines)
                for dt in dates:
                    if count > 6 and count <= 16:      
                        Published.append(dt[3:])
                break
    
        df_start_end = pd.DataFrame({"Headlines":headlines,'Published_on':Published})
#         return(df_start_end)    

#These files will be stored in the specified folder when we will be executing the multithreading block
    file = open("pushkar_output/page"+"_"+str(start)+"_to_"+str(end), 'wb')

    # dump information to that file
    pickle.dump(df_start_end, file)

    # close the file
    file.close()


In [2]:
import threading

In [3]:
threadlist = []
#We provide batches for eg:here we see a batch of 20 pages
#Depending on our system config we can tweak this
for i in range(451,880,20):
#     print(i,i+20)
    t = threading.Thread(target=web_scrape , args = ("https://www.investing.com/equities/google-inc-c-news/",i,i+20,))
    threadlist.append(t)
    t.start()
    
for tr in threadlist:
    tr.join()
    print("done")
    

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [4]:
import os

Now we need to open our output pickle files.

We see that each file conatins a list of Dataframes corresponding to respective batch.

The last step is to merge output from all the files, manipulating the dataframe and exporting to csv.

In [5]:
allcontacts = []

for root, dirs, files in os.walk("pushkar_output/"):
    for f in files:
#         print(f)
#         f = files
        opencontacts = open('C:/Users/pushk/Notebooks/pushkar_output/' + f,'rb')
        loadedcontacts = pickle.load(opencontacts)
#         print(loadedcontacts)
        allcontacts.append(loadedcontacts)

In [6]:
final_df = pd.DataFrame()
for i in range(len(allcontacts)):
    df = final_df.append(allcontacts[i]).reset_index(drop = True)
    final_df = df
#     print(final_df)

In [7]:
final_df['Published_on'] = pd.to_datetime(final_df['Published_on'])


In [8]:
final_df = final_df.sort_values(by=['Published_on'],ascending=False)
final_df.reset_index(inplace=True,drop=True)
final_df

,Headlines,Published_on
0,Exclusive: U.S. agency probes Facebook for 'sy...,2021-03-06
1,"Fueled by tech, Wall Street rebounds at end of...",2021-03-06
2,S&P 500 Snaps 2-Week Losing Streak as Dip Buyi...,2021-03-06
3,Biden adds Big Tech critic Tim Wu to his econo...,2021-03-06
4,Hackers target Williams F1 livery launch on AR...,2021-03-06
...,...,...
8795,"Indian firms could get boost from Google, Alibaba",2015-03-12
8796,Rocket Internet-backed Helpling expands in Asi...,2015-03-12
8797,"DreamWorks, AwesomenessTV to make original sho...",2015-03-12
8798,Wall St. rallies; Nasdaq hits highest level si...,2015-03-12


In [10]:
final_df.to_csv('final_goog_headlines1.csv')